In [1]:
from torchmetrics.functional import auc, mean_squared_error
from torchmetrics import F1Score
from tools import *
from CONSTANT import *
from models import CNNBiLSTM, CNNTransformer
from config import Params
from torch.utils.data import (
    TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import time
import datetime
import os
import sys
import time
import warnings

# load baseline results

In [2]:
bs_results = load_dict_model(r'./output/KEC/valence_CTransformer_loso_0.0001_64_32/results.pkl')
parse_res(bs_results)
# valence CTransformer

0.56804

In [3]:
bs_results = load_dict_model(r'./output/KEC/arousal_CLSTM_loso_0.0001_64_32/results.pkl')
parse_res(bs_results)
# valence CLSTM

0.9817

# set params

In [3]:
ckpt_path = r'./output/HKU956/valence_CTransformer_loso_0.0001_256_32/fold1_checkpoint.pt'

args = Params(dataset='KEC', 
              model='CTransformer',
              target='valence', 
              debug=False, 
              fcn_input=12608,
              batch_size=32
              )

# load data

In [4]:
spliter = load_model(args.spliter)
data = pd.read_pickle(args.data)

for i, k in enumerate(spliter[args.valid]):
    train_index = k['train_index']
    test_index = k['test_index']
    break

dataprepare = DataPrepare(args,
                        target=args.target, data=data, train_index=train_index, test_index=test_index, device=args.device, batch_size=args.batch_size
                        )

train_dataloader, test_dataloader = dataprepare.get_data()

(2837, 4, 400) (2837, 1) (608, 4, 400) (608, 1)


# load pretrain model

In [5]:
kec_fcn = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(args.fcn_input, 128),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

In [6]:
model = CNNTransformer.CTransformer(args)
model.load_state_dict(torch.load(ckpt_path))
model.fcn = kec_fcn
model = model.to(args.device)

# train and eval

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss_list = []
    loss_fn = nn.MSELoss()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target.float())
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
    return np.mean(train_loss_list)


def eval(model, device, val_loader):
    model.eval()
    val_loss = []
    loss_fn = nn.MSELoss()
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target.float())
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [8]:
def run(train_loader, val_loader, ckpt_path):
    best_score = float('inf')
    patience = 25
    stop_count = 0
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                                    verbose=True, threshold_mode='rel',
                                    cooldown=0, min_lr=0, eps=1e-08
                                    )
    for epoch in range(1, args.epochs + 1):
        train_loss = train(model, args.device, train_loader, optimizer, epoch)
        val_loss = eval(model, args.device, val_loader)
        scheduler.step(val_loss)
        print('[Epoch{}] | train_loss:{:.4f} | val_loss:{:.4f} | lr:{:e}'.format(epoch, train_loss, val_loss, optimizer.param_groups[0]['lr']))

        if val_loss < best_score:
            best_score = val_loss
            torch.save(model.state_dict(), ckpt_path)
            print("<<<<<< reach best {0} >>>>>>".format(val_loss))
            stop_count = 0
        else:
            model.load_state_dict(torch.load(ckpt_path))
            stop_count += 1
            if stop_count >= patience:
                print("<<<<<< without improvement in {} epoch, early stopping, best score {:.4f} >>>>>>".format(patience, best_score))
                break
        # wandb.log({'train_loss': train_loss, 'val_loss': val_loss})
    print('best score', best_score)
    return model

In [9]:
model = run(train_dataloader, test_dataloader, ckpt_path=os.path.join(args.save_path, 'checkpoint.pt'))

89it [00:05, 15.39it/s]


[Epoch1] | train_loss:1.3936 | val_loss:0.5025 | lr:1.000000e-04
<<<<<< reach best 0.5025444182346722 >>>>>>


89it [00:03, 23.02it/s]


[Epoch2] | train_loss:0.7854 | val_loss:0.5155 | lr:1.000000e-04


89it [00:03, 23.08it/s]


...

89it [00:03, 23.19it/s]


Epoch 00052: reducing learning rate of group 0 to 7.8125e-07.
[Epoch52] | train_loss:0.5915 | val_loss:0.4804 | lr:7.812500e-07


89it [00:03, 23.21it/s]


[Epoch53] | train_loss:0.5997 | val_loss:0.4745 | lr:7.812500e-07
<<<<<< without improvement in 25 epoch, early stopping, best score 0.4706 >>>>>>


CTransformer(
  (cnns): Sequential(
    (0): Conv1d(4, 16, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
  )
  (transformer): TransformerBlock(
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder_layers): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
      )
      (linear1): Linear(in_features=32, out_features=2048, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (linear2): Linear(in_features=204